In [1]:
import sys
sys.path.append('../')
import json
import pandas as pd
import sqlite3
from helper import load_dotenv
from tqdm.auto import tqdm
from pathlib import Path

In [2]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [3]:
pages = pd.read_sql(
    '''
        SELECT * FROM generic_image gi 
        WHERE gi.job = '24-03-05-01_randomTrainPages'
    ''',
    con
)

adjustments_per_page = pd.read_sql(
    '''
        SELECT apm.*, m.pattern, m.overlay_intensity_K FROM adjustment_per_mask apm 
        LEFT JOIN mask m
        ON apm.job=m.job AND apm.pdf_filename=m.pdf_filename AND apm."type"=m."type" AND apm.variant_name=m.variant_name AND apm.method=m.method AND apm.idx=m.idx AND apm.mask_id=m.mask_id
        WHERE apm.job = '24-03-05-01_randomTrainPages'
    ''',
    con
)

In [82]:
configs = {}

for config_path in Path('../../60_gebastel/Musterueberlagerung/configurations/').glob('./*.json'):
    with config_path.open() as config_file:
        data = json.load( config_file )
        configs[data['config_name']] = data

In [63]:
def check_config( config, adjustments_of_page, debug=False ):
    def log( message ):
        if debug:
            print( message )
    
    # check pattern
    if False in adjustments_of_page.pattern.isin( config['available_patterns'] ):
        log( 'pattern not available' )
        return False

    # check overlay intensity
    if False in adjustments_of_page.overlay_intensity_K.between(config['overlay_weight_min'], config['overlay_weight_max']):
        log( f"overlay weight not between {config['overlay_weight_min']} and {config['overlay_weight_max']}" )
        return False

    # check for weight=0 flaw
    for key in config['adjustments']:
        if config['adjustments'][key]['weight'] == 0 and adjustments_of_page[adjustments_of_page.adjustment == key].shape[0] > 0:
            log( f"{ key } not an available adjustment" )
            return False

    # scale
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'scale'].iterrows():
        features = json.loads( row.features )

        if (config['adjustments']['scale']['min_size'] <= features['scale'] <= config['adjustments']['scale']['max_size']) == False:
            log( "scale: value not in range" )
            return False

    
    # rotation
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'rotation'].iterrows():
        features = json.loads( row.features )
        if features['rotation_degree'] < config['adjustments']['rotation']['base_degree'] - config['adjustments']['rotation']['degree_span']:
            log( "rotation degree too low" )
            return False
    
        if features['rotation_degree'] > config['adjustments']['rotation']['base_degree'] + config['adjustments']['rotation']['degree_span']:
            log( "rotation degree too high" )
            return False

    # blow_up centered
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'blow_up_centered'].iterrows():
        features = json.loads( row.features )
        if features['centered_c'] != config['adjustments']['blow_up_centered']['c']:
            log( f"blow_up_centered - c missmatch: { features['centered_c'] }/{ config['adjustments']['blow_up_centered']['c'] }" )
            return False

    # contract_centered
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'contract_centered'].iterrows():
        features = json.loads( row.features )
        if features['centered_c'] != config['adjustments']['contract_centered']['c']:
            log( f"contract_centered - c missmatch: { features['centered_c'] }/{ config['adjustments']['blow_up_centered']['c'] }" )
            return False

    # blow_up_region
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'blow_up_region'].iterrows():
        features = json.loads( row.features )
    
        if (config['adjustments']['blow_up_region']['min_count'] <= features['blow_up_count'] <= config['adjustments']['blow_up_region']['max_count']) == False:
            log( f"blow_up_region: count not in range" )
            return False
    
    
        for r in features['blow_up_radius']:
            if r < config['adjustments']['blow_up_region']['min_radius']:
                return False
        
            if r > config['adjustments']['blow_up_region']['max_radius']:
                return False
    
        for c in features['blow_up_c']:
            if c != config['adjustments']['blow_up_region']['c']:
                return False


    # contract_region
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'contract_region'].iterrows():
        features = json.loads( row.features )
    
        if features['contract_count'] < config['adjustments']['contract_region']['min_count']:
            return False
    
        if features['contract_count'] > config['adjustments']['contract_region']['max_count']:
            return False
    
        for r in features['contract_radius']:
            if r < config['adjustments']['contract_region']['min_radius']:
                return False
        
            if r > config['adjustments']['contract_region']['max_radius']:
                return False
    
        for c in features['contract_c']:
            if c != config['adjustments']['contract_region']['c']:
                return False
    
    # stretch
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'stretch'].iterrows():
        features = json.loads( row.features )
    
        if (config['adjustments']['stretch']['min_stretch_x'] <= features['stretch_x'] <= config['adjustments']['stretch']['max_stretch_x']) == False:
            log( "stretch-x not in range" )
            return False
    
        if (config['adjustments']['stretch']['min_stretch_y'] <= features['stretch_y'] <= config['adjustments']['stretch']['max_stretch_y']) == False:
            log( "stretch-y not in range" )
            return False    

    # trapezoidal_distortion
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'trapezoidal_distortion'].iterrows():
        corner_adjustments = json.loads( row.features )['corner_adjustments']
        distortion_settings = config['adjustments']['trapezoidal_distortion']
    
        if (distortion_settings['top_left']['x']['min'] <= corner_adjustments[0][0] <= distortion_settings['top_left']['x']['max']) == False:
            log("trapezoidal_distortion - top-left x not in range")
            return False
    
        if (distortion_settings['top_left']['y']['min'] <= corner_adjustments[0][1] <= distortion_settings['top_left']['y']['max']) == False:
            log("trapezoidal_distortion - top-left y not in range")
            return False
    
        if (distortion_settings['bottom_left']['x']['min'] <= corner_adjustments[1][0] <= distortion_settings['bottom_left']['x']['max']) == False:
            log("trapezoidal_distortion - bottom_left x not in range")
            return False
    
        if (distortion_settings['bottom_left']['y']['min'] <= corner_adjustments[1][1] <= distortion_settings['bottom_left']['y']['max']) == False:
            log("trapezoidal_distortion - bottom_left y not in range")
            return False
    
        if (distortion_settings['bottom_right']['x']['min'] <= corner_adjustments[2][0] <= distortion_settings['bottom_right']['x']['max']) == False:
            log("trapezoidal_distortion - bottom_right x not in range")
            return False
    
        if (distortion_settings['bottom_right']['y']['min'] <= corner_adjustments[2][1] <= distortion_settings['bottom_right']['y']['max']) == False:
            log("trapezoidal_distortion - bottom_right y not in range")
            return False
    
        if (distortion_settings['top_right']['x']['min'] <= corner_adjustments[3][0] <= distortion_settings['top_right']['x']['max']) == False:
            log("trapezoidal_distortion - top_right x not in range")
            return False
    
        if (distortion_settings['top_right']['y']['min'] <= corner_adjustments[3][1] <= distortion_settings['top_right']['y']['max']) == False:
            log("trapezoidal_distortion - top_right y not in range")
            return False

    # uniform_trapezoidal_distortion
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'uniform_trapezoidal_distortion'].iterrows():
        features = json.loads( row.features )
    
        direction_index = ['left','right','top','bottom'].index(features['trapezoidal_distortion_direction'])
        if config['adjustments']['uniform_trapezoidal_distortion']['chance_directions'][direction_index] == 0:
            log('uniform_trapezoidal_distortion: direction chance is 0')
            return False
    
        if type(features['trapezoidal_distortion_strength']) != list:
            log('uniform_trapezoidal_distortion: strength is not 2d')
            return False
    
        if (config['adjustments']['uniform_trapezoidal_distortion']['x']['min'] <= features['trapezoidal_distortion_strength'][0] <= config['adjustments']['uniform_trapezoidal_distortion']['x']['max']) == False:
            log('uniform_trapezoidal_distortion: x value out of range')
            return False
    
        if (config['adjustments']['uniform_trapezoidal_distortion']['y']['min'] <= features['trapezoidal_distortion_strength'][1] <= config['adjustments']['uniform_trapezoidal_distortion']['y']['max']) == False:
            log('uniform_trapezoidal_distortion: y value out of range')
            return False

    # uniform_trapezoidal_distortion
    for idx,row in adjustments_of_page.loc[adjustments_of_page.adjustment == 'wave_deform'].iterrows():
        features = json.loads( row.features )
    
        if (config['adjustments']['wave_deform']['min_wave_overlay_count'] <= features['wave_overlay_count'] <= config['adjustments']['wave_deform']['max_wave_overlay_count']) == False:
            log('wave_deform: wave_overlay_count out of range')
            return False
    
        for wave_config in features['wave_configurations']:
            if (config['adjustments']['wave_deform']['min_wave_length'] <= wave_config[0] <= config['adjustments']['wave_deform']['max_wave_length']) == False:
                log('wave_deform: wave_length out of range')
                return False
    
            if (config['adjustments']['wave_deform']['min_wave_depth'] <= wave_config[1] <= config['adjustments']['wave_deform']['max_wave_depth']) == False:
                log('wave_deform: wave_depth out of range')
                return False

    return True


def get_possible_configs( page, adjustments_per_page, configs, debug=False ):
    relevant_adjustments = adjustments_per_page.loc[
        (adjustments_per_page.job==page.job) &
        (adjustments_per_page.pdf_filename==page.pdf_filename) &
        (adjustments_per_page.type==page.type) &
        (adjustments_per_page.variant_name==page.variant_name) &
        (adjustments_per_page.method==page.method) &
        (adjustments_per_page.idx==page.idx)
    ]

    #return relevant_adjustments
    possible_configs = []

    for config_name in configs:
        if debug:
            print( f'checking: { config_name }')
        
        if check_config( configs[config_name], relevant_adjustments,debug=debug ):
            possible_configs.append( config_name )

    return possible_configs

In [89]:
possible_configs = []

for idx,page in pages.iterrows():
    try:
        possible_configs.append((
            idx,
            get_possible_configs( page, adjustments_per_page, configs )
        ))
    except:
        pass

In [91]:
possible_configs[-10:]

[(771, ['rotate']),
 (772, ['rotate']),
 (773, ['general']),
 (774, ['rotate', 'deform']),
 (775, ['rotate', 'deform']),
 (776, ['rotate', 'deform']),
 (777, ['rotate', 'deform']),
 (778, ['rotate']),
 (779, ['rotate', 'deform']),
 (780, ['stretch'])]

In [92]:
pages.iloc[-10:]

,pdf_filename,job,type,variant_name,method,idx,timestamp,config_name
771,149714.IO43_gmg-konv43_9783060113415 Inhalt_S1...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:19:55.720817,deform
772,149958.tg51_TMM0224S068.p1,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:19:57.306903,deform
773,136209.g47_101-200_The-Patch.p44,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,3,2024-04-30 09:19:58.429173,general
774,510536.001_2024_007_0_098_0.p1,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:19:59.401007,deform
775,149359.IO43_gmg-konv43_9783464850435 Inhalt_S1...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:20:00.104408,deform
776,123032.074-075_Chronik-1941.p1,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:20:01.210000,deform
777,142234.g39_DO01_3-12-548341_Inhalt_S001-216.p3,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,2,2024-04-30 09:20:02.118876,deform
778,140586.g47_149_001-200_EDEKA_Innenteilselbstge...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:20:03.525153,deform
779,150017.tg39_HDESWM072_009_040.p24,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:20:05.414612,deform
780,149261.IO43_gmg-konv43_9783464812044__064.p1,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-04-30 09:20:06.464636,stretch


In [93]:
page = pages.loc[771]

page

pdf_filename    149714.IO43_gmg-konv43_9783060113415 Inhalt_S1...
job                                  24-03-05-01_randomTrainPages
type                                                           4c
variant_name                                       halftone600dpi
method                                                 soft_light
idx                                                             1
timestamp                              2024-04-30 09:19:55.720817
config_name                                                deform
Name: 771, dtype: object

In [94]:
adjustments_of_page = adjustments_per_page.loc[
    (adjustments_per_page.job==page.job) &
    (adjustments_per_page.pdf_filename==page.pdf_filename) &
    (adjustments_per_page.type==page.type) &
    (adjustments_per_page.variant_name==page.variant_name) &
    (adjustments_per_page.method==page.method) &
    (adjustments_per_page.idx==page.idx) &
    (adjustments_per_page.adjustment == 'trapezoidal_distortion')
]

In [95]:
get_possible_configs(
    page,
    adjustments_per_page,
    configs,
    debug=True
)

checking: rotate
checking: general
rotation degree too high
checking: deform
blow_up_region not an available adjustment
checking: pullover
rotation degree too low
checking: stretch
trapezoidal_distortion not an available adjustment


['rotate']